In [1]:
import pandas as pd
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset
from torch.nn import CrossEntropyLoss
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import torch.nn as nn
import matplotlib.pyplot as plt
from kobert_transformers import get_kobert_model
from kobert_transformers.tokenization_kobert import KoBertTokenizer
import numpy as np
import time
from matplotlib import rcParams
# 한글 폰트 설정
rcParams['font.family'] = 'Malgun Gothic'  # 맑은 고딕으로 설정
rcParams['axes.unicode_minus'] = False     # 음수 기호 문제 해결

In [2]:
df_bigkinds = pd.read_excel('./Text_Data/bigkinds/삼성전자_정치_20170901-20240830_전처리후.xlsx', converters={'뉴스 식별자': str})  # '뉴스 식별자'는 해당 열의 실제 이름으로 대체

In [3]:
df_bigkinds

,뉴스 식별자,일자,언론사,기고자,제목,통합 분류1,통합 분류2,통합 분류3,사건/사고 분류1,사건/사고 분류2,...,URL,분석제외 여부,제목_키워드,제목_키워드_전처리후,제목_키워드_전처리후_Okt,제목_키워드_전처리후_Hannanum,제목_키워드_전처리후_Kkma,제목_키워드_전처리후_Komoran,제목_키워드_전처리후_kiwi,제목_키워드_전처리후_Komoran_명사
0,02100101.20170901174354001,2017-09-01,매일경제,김제이,최순실게이트 청문회 불출석 11인 중 8명 `백수`,사회>사건_사고,정치>청와대,정치>국회_정당,범죄>정치>뇌물수수,NaN,...,http://news.mk.co.kr/newsRead.php?year=2017&no...,NaN,"최순실게이트 청문회 불출석 11인 중 8명 `백수` 불출석,최순실게이트,청문회,백수...",최순실게이트 청문회 불출석 11인 중 8명 백수 불출석 최순실게이트 청문회 백수 문...,최 순 실 게이트 청문회 불 출석 11 인 중 8 명 백수 불 출석 최 순 실 게이...,최순실게이트 청문회 불출석 11 ㄴ 중 8명 백수 불출석 최순실게이트 청문회 백수 ...,최 순 실 게이트 청문회 불 출석 11 인 중 8 명 백 수 불 출석 최 순 실 게...,최순 실 게이트 청문회 불 출석 11 ㄴ 중 8 명 백수 불 출석 최순 실 게이트 ...,최순실 게이트 청문회 출석 11 백수 출석 최순실 게이트 청문회 백수 문고리 안봉근...,최순 실 게이트 청문회 출석 중 명 백수 출석 최순 실 게이트 청문회 백수 문고리 ...
1,02100101.20170901141935001,2017-09-01,매일경제,정주원,두문불출 10개월만에 `문고리 실세` 이재만 안봉근 법정 나와 청문회불출석 인정,사회>사건_사고,정치>청와대,정치>국회_정당,NaN,NaN,...,http://news.mk.co.kr/newsRead.php?year=2017&no...,NaN,두문불출 10개월만에 `문고리 실세` 이재만 안봉근 법정 나와 청문회불출석 인정 두...,두문불출 10개월만에 문고리 실세 이재만 안봉근 법정 나와 청문회불출석 인정 두문불...,두문불출 10 개월 만에 문 고리 실세 이재만 안봉 근 법정 나와 청문회 불 출석 ...,두문불출 10개월 만에 문고리 실세 이재 만 안봉근 법정 청문회불출석 인정 두문불출...,두문불출 10 개월 만 문고리 실세 이재 만 알 ㄴ 봉 근 법정 나오 청문회 불 출...,두문불출 10 개월 만 문고리 실세 이재만 안 봉 근 법정 나오 청문회 불 출석 인...,두문불출 10 개월 문고리 실세 이재만 안봉근 법정 청문회 출석 인정 두문불출 개월...,두문불출 개월 문고리 실세 이재만 안 봉 근 법정 청문회 출석 인정 두문불출 개월 ...
2,02100311.20170901153936001,2017-09-01,서울경제,장주영 기자,"청문회 불출석 혐의, 건강상태-절차적 하자로 '무죄' 주장",정치>국회_정당,사회>사건_사고,정치>청와대,NaN,NaN,...,http://www.sedaily.com/NewsView/1OKUY3ELL1,NaN,"청문회 불출석 혐의, 건강상태-절차적 하자로 '무죄' 주장 혐의,청문회,건강상태,절...",청문회 불출석 혐의 건강상태-절차적 하자로 무죄 주장 혐의 청문회 건강상태 절차적 ...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 불출석 혐의 건강상태 - 절차적 하자 무죄 주장 혐의 청문회 건강상태 절차적...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 출석 혐의 건강 상태 절차 하자 주장 혐의 청문회 건강 상태 절차 하자 무죄...,청문회 출석 혐의 건강 상태 절차 하자 무죄 주장 혐의 청문회 건강 상태 절차 무죄...
3,02100101.20170902162415001,2017-09-02,매일경제,디지털뉴스국,서울 도심 곳곳서 태극기 집회,정치>청와대,정치>행정_자치,정치>선거,사회>사회갈등>시위,NaN,...,http://news.mk.co.kr/newsRead.php?year=2017&no...,NaN,"서울 도심 곳곳서 태극기 집회 서울,도심,태극기,집회,주말,도심,서울,도심,박근혜,...",서울 도심 곳곳서 태극기 집회 서울 도심 태극기 집회 주말 도심 서울 도심 박근혜 ...,서울 도심 곳곳 서 태극기 집회 서울 도심 태극기 집회 주말 도심 서울 도심 박근혜...,서울 돌 시ㅁ 곳곳 서 태극기 집회 서울 도 시ㅁ 태극기 집회 주말 도 시ㅁ 서울 ...,서울 도심 곳곳 서 태극기 집회 서울 도심 태극기 집회 주말 도심 서울 도심 박근 ...,서울 도심 곳곳 서 태극기 집회 서울 도심 태극기 집회 주말 도심 서울 도심 박근혜...,서울 도심 곳곳 태극기 집회 서울 도심 태극기 집회 주말 도심 서울 도심 박근혜 단...,서울 도심 곳곳 태극기 집회 서울 도심 태극기 집회 주말 도심 서울 도심 박근혜 단...
4,02100101.20170904091746001,2017-09-04,매일경제,고득관,"코스피, 북핵 리스크에 2330선 `털썩`",정치>북한,정치>청와대,NaN,범죄>기업범죄>거래제한,재해>자연재해>지진,...,http://news.mk.co.kr/newsRead.php?year=2017&no...,NaN,"코스피, 북핵 리스크에 2330선 `털썩` 코스피,리스크,2330선,북한,지정학,리...",코스피 북핵 리스크에 2330선 털썩 코스피 리스크 2330선 북한 지정학 리스크 ...,코스피 북핵 리스크 2330 선 털썩 코스피 리스크 2330 선 북한 지정학 리스크...,코스피 북핵 리스크 2330 선 털썩 코스피 리스크 2330 선 북한 지정학 리스크...,코스 피 북 핵 리스크 2330 선 털썩 코스 피 리스크 2330 선 북 ㄴ 지정학...,코스 피 북 핵 리스크 2330 선 털썩 코스 피 리스크 2330 선 북한 지정학 ...,코스피 북핵 리스크 2330 코스피 리스크 2330 북한 지정학 리스크 고조 코스피...,코스 피 북 핵 리스크 선 코스 피 리스크 선 북한 지정학 리스크 고조 코스 피 급...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5407,02100101.20240829144509001,2024-08-29,매일경제,이호준 매경이코노미 인턴기자(lhj0756@naver.com),재산 333억원에 페라리 소유...국회의원 재산신고 1위 누구?,정치>국회_정당,정치>선거,NaN,NaN,NaN,...,http://www.mk.co.kr/article/11104605,NaN,"재산 333억원에 페라리 소유...국회의원 재산신고 1위 누구? 재산,페라리,소유,...",재산 333억원에 페라리 소유...국회의원 재산신고 1위 누구 재산 페라리 소유 국...,재산 333억원 페라리 소유 국회의원 재산 신고 1 위 재산 페라리 소유 국회의원 ...,재산 333억원 페라리 소유 국회의원 재산신고 1위 재산 페라리 소유 국회의원 재산...,재산 333 억 원 페라리 소유 국회 의원 재산 신고 1 위 누 재산 페라리 소유 ...,재산 333 억 원 페라리 소유 국회의원 재산 신고 1 위 재산 페라리 소유 국회의...,재산 333 페라리 소유 국회 의원 재산 신고 재산 페라리 소유 국회 의원 재산 신...,재산 원 페라리 소유 국회의원 재산 신고 위 재산 페라리 소유 국회의원 재산 신고 ...
5408,02100311.20240829000230001,2024-08-29,서울경제,김예솔 기자,신규 국회의원 평균재산 27억 고동진 333억 '최고',정치>국회_정당,정치>선거,NaN,NaN,NaN,...,http://www.sedaily.com/NewsView/2DD7CNUYYS,NaN,"신규 국회의원 평균재산 27억 고동진 333억 '최고' 신규,국회의원,평균,재산,고...",신규 국회의원 평균재산 

In [4]:
# KoBERT 모델 및 토크나이저 로드
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')
model = get_kobert_model()

In [5]:
# 감정 분석을 위한 회귀 레이어 추가
class KoBERTSentimentRegressor(nn.Module):
    def __init__(self, base_model, output_size=1):  # output_size를 1로 설정
        super(KoBERTSentimentRegressor, self).__init__()
        self.base_model = base_model
        self.regressor = nn.Linear(base_model.config.hidden_size, output_size)  # 연속적인 출력값을 위한 선형 레이어
    
    def forward(self, input_ids, attention_mask):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output  # [CLS] 토큰의 출력
        return self.regressor(pooled_output)

# 감정 분석 모델 생성
regressor = KoBERTSentimentRegressor(model)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
regressor.to(device)

KoBERTSentimentRegressor(
  (base_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [6]:
# 데이터셋 클래스 정의
class SentimentDataset(Dataset):
    def __init__(self, texts, tokenizer, max_len):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        input_ids = encoding['input_ids'].squeeze(0)
        attention_mask = encoding['attention_mask'].squeeze(0)
        
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask
        }

# 데이터셋 준비
MAX_LEN = 64  # 최대 토큰 길이 설정
dataset = SentimentDataset(df_bigkinds['제목_키워드_전처리후_Komoran'].values, tokenizer, MAX_LEN)
data_loader = DataLoader(dataset, batch_size=16)

In [7]:
# 감정 분석 예측 함수
def predict_sentiments(model, data_loader, device):
    model.eval()
    predictions = []
    
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            sentiment_scores = model(input_ids=input_ids, attention_mask=attention_mask)
            scores = sentiment_scores.cpu().numpy().flatten()
            # -1 ~ 1 범위로 클리핑
            scores = np.clip(scores, -1, 1)
            predictions.extend(scores)
    
    return predictions

# 감정 분석 수행
s = time.time()
predicted_scores = predict_sentiments(regressor, data_loader, device)
print("Prediction Time: ", time.time()-s)

C:\Users\dybae\anaconda3\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Prediction Time:  12.703468561172485


In [8]:
# 뉴스 식별자 컬럼에서 '.' 이후의 값만 조회
df_bigkinds['뉴스 식별자'] = df_bigkinds['뉴스 식별자'].str.split('.').str[1]

In [9]:
# 데이터프레임에 결과 추가
df_bigkinds['Bert_Sentiment'] = predicted_scores
# 소수점 1자리로 반올림
df_bigkinds['Bert_Sentiment'] = df_bigkinds['Bert_Sentiment'].round(1)
df_bigkinds['Bert_Sentiment'] = df_bigkinds['Bert_Sentiment'].apply(lambda x: 0 if x == -0.0 else x)
# 결과 확인
df_bigkinds

,뉴스 식별자,일자,언론사,기고자,제목,통합 분류1,통합 분류2,통합 분류3,사건/사고 분류1,사건/사고 분류2,...,분석제외 여부,제목_키워드,제목_키워드_전처리후,제목_키워드_전처리후_Okt,제목_키워드_전처리후_Hannanum,제목_키워드_전처리후_Kkma,제목_키워드_전처리후_Komoran,제목_키워드_전처리후_kiwi,제목_키워드_전처리후_Komoran_명사,Bert_Sentiment
0,20170901174354001,2017-09-01,매일경제,김제이,최순실게이트 청문회 불출석 11인 중 8명 `백수`,사회>사건_사고,정치>청와대,정치>국회_정당,범죄>정치>뇌물수수,NaN,...,NaN,"최순실게이트 청문회 불출석 11인 중 8명 `백수` 불출석,최순실게이트,청문회,백수...",최순실게이트 청문회 불출석 11인 중 8명 백수 불출석 최순실게이트 청문회 백수 문...,최 순 실 게이트 청문회 불 출석 11 인 중 8 명 백수 불 출석 최 순 실 게이...,최순실게이트 청문회 불출석 11 ㄴ 중 8명 백수 불출석 최순실게이트 청문회 백수 ...,최 순 실 게이트 청문회 불 출석 11 인 중 8 명 백 수 불 출석 최 순 실 게...,최순 실 게이트 청문회 불 출석 11 ㄴ 중 8 명 백수 불 출석 최순 실 게이트 ...,최순실 게이트 청문회 출석 11 백수 출석 최순실 게이트 청문회 백수 문고리 안봉근...,최순 실 게이트 청문회 출석 중 명 백수 출석 최순 실 게이트 청문회 백수 문고리 ...,-0.1
1,20170901141935001,2017-09-01,매일경제,정주원,두문불출 10개월만에 `문고리 실세` 이재만 안봉근 법정 나와 청문회불출석 인정,사회>사건_사고,정치>청와대,정치>국회_정당,NaN,NaN,...,NaN,두문불출 10개월만에 `문고리 실세` 이재만 안봉근 법정 나와 청문회불출석 인정 두...,두문불출 10개월만에 문고리 실세 이재만 안봉근 법정 나와 청문회불출석 인정 두문불...,두문불출 10 개월 만에 문 고리 실세 이재만 안봉 근 법정 나와 청문회 불 출석 ...,두문불출 10개월 만에 문고리 실세 이재 만 안봉근 법정 청문회불출석 인정 두문불출...,두문불출 10 개월 만 문고리 실세 이재 만 알 ㄴ 봉 근 법정 나오 청문회 불 출...,두문불출 10 개월 만 문고리 실세 이재만 안 봉 근 법정 나오 청문회 불 출석 인...,두문불출 10 개월 문고리 실세 이재만 안봉근 법정 청문회 출석 인정 두문불출 개월...,두문불출 개월 문고리 실세 이재만 안 봉 근 법정 청문회 출석 인정 두문불출 개월 ...,0.0
2,20170901153936001,2017-09-01,서울경제,장주영 기자,"청문회 불출석 혐의, 건강상태-절차적 하자로 '무죄' 주장",정치>국회_정당,사회>사건_사고,정치>청와대,NaN,NaN,...,NaN,"청문회 불출석 혐의, 건강상태-절차적 하자로 '무죄' 주장 혐의,청문회,건강상태,절...",청문회 불출석 혐의 건강상태-절차적 하자로 무죄 주장 혐의 청문회 건강상태 절차적 ...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 불출석 혐의 건강상태 - 절차적 하자 무죄 주장 혐의 청문회 건강상태 절차적...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 출석 혐의 건강 상태 절차 하자 주장 혐의 청문회 건강 상태 절차 하자 무죄...,청문회 출석 혐의 건강 상태 절차 하자 무죄 주장 혐의 청문회 건강 상태 절차 무죄...,-0.2
3,20170902162415001,2017-09-02,매일경제,디지털뉴스국,서울 도심 곳곳서 태극기 집회,정치>청와대,정치>행정_자치,정치>선거,사회>사회갈등>시위,NaN,...,NaN,"서울 도심 곳곳서 태극기 집회 서울,도심,태극기,집회,주말,도심,서울,도심,박근혜,...",서울 도심 곳곳서 태극기 집회 서울 도심 태극기 집회 주말 도심 서울 도심 박근혜 ...,서울 도심 곳곳 서 태극기 집회 서울 도심 태극기 집회 주말 도심 서울 도심 박근혜...,서울 돌 시ㅁ 곳곳 서 태극기 집회 서울 도 시ㅁ 태극기 집회 주말 도 시ㅁ 서울 ...,서울 도심 곳곳 서 태극기 집회 서울 도심 태극기 집회 주말 도심 서울 도심 박근 ...,서울 도심 곳곳 서 태극기 집회 서울 도심 태극기 집회 주말 도심 서울 도심 박근혜...,서울 도심 곳곳 태극기 집회 서울 도심 태극기 집회 주말 도심 서울 도심 박근혜 단...,서울 도심 곳곳 태극기 집회 서울 도심 태극기 집회 주말 도심 서울 도심 박근혜 단...,-0.1
4,20170904091746001,2017-09-04,매일경제,고득관,"코스피, 북핵 리스크에 2330선 `털썩`",정치>북한,정치>청와대,NaN,범죄>기업범죄>거래제한,재해>자연재해>지진,...,NaN,"코스피, 북핵 리스크에 2330선 `털썩` 코스피,리스크,2330선,북한,지정학,리...",코스피 북핵 리스크에 2330선 털썩 코스피 리스크 2330선 북한 지정학 리스크 ...,코스피 북핵 리스크 2330 선 털썩 코스피 리스크 2330 선 북한 지정학 리스크...,코스피 북핵 리스크 2330 선 털썩 코스피 리스크 2330 선 북한 지정학 리스크...,코스 피 북 핵 리스크 2330 선 털썩 코스 피 리스크 2330 선 북 ㄴ 지정학...,코스 피 북 핵 리스크 2330 선 털썩 코스 피 리스크 2330 선 북한 지정학 ...,코스피 북핵 리스크 2330 코스피 리스크 2330 북한 지정학 리스크 고조 코스피...,코스 피 북 핵 리스크 선 코스 피 리스크 선 북한 지정학 리스크 고조 코스 피 급...,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5407,20240829144509001,2024-08-29,매일경제,이호준 매경이코노미 인턴기자(lhj0756@naver.com),재산 333억원에 페라리 소유...국회의원 재산신고 1위 누구?,정치>국회_정당,정치>선거,NaN,NaN,NaN,...,NaN,"재산 333억원에 페라리 소유...국회의원 재산신고 1위 누구? 재산,페라리,소유,...",재산 333억원에 페라리 소유...국회의원 재산신고 1위 누구 재산 페라리 소유 국...,재산 333억원 페라리 소유 국회의원 재산 신고 1 위 재산 페라리 소유 국회의원 ...,재산 333억원 페라리 소유 국회의원 재산신고 1위 재산 페라리 소유 국회의원 재산...,재산 333 억 원 페라리 소유 국회 의원 재산 신고 1 위 누 재산 페라리 소유 ...,재산 333 억 원 페라리 소유 국회의원 재산 신고 1 위 재산 페라리 소유 국회의...,재산 333 페라리 소유 국회 의원 재산 신고 재산 페라리 소유 국회 의원 재산 신...,재산 원 페라리 소유 국회의원 재산 신고 위 재산 페라리 소유 국회의원 재산 신고 ...,0.0
5408,20240829000230001,2024-08-29,서울경제,김예솔 기자,신규 국회의원 평균재산 27억 고동진 333억 '최고',정치>국회_정당,정치>선거,NaN,NaN,NaN,...,NaN,"신규 국회의원 평균재산 27억 고동진 333억 '최고' 신규,국회의원,평균,재산,고...",신규 국회의원 평균재산 27억 고동진 333억 최고 신규 국회의원 평균 재산 고동진...,신규 국회의원 평균 재산 27억 고동진 333억 최고 신규 국회의원 평균 재산 고동...,신규 국회의원 평균재산 27억 고동진 333억 최고 신규 국회의원 평균 재산 고동진...,신규 국회의원 평균 재산 27 억 고동진 333 억 최고 신규 국회의원 평균 재산 ...,신규 국회의원 평균 재산 27 억 고동진 333 억 최고 신규 국회의원 평균 재산 ...,신규 국회의원 평균 재산 27 고동진 333 최고 신규 국회의원 평균 재산 고동진 ...,신규 국회의원 평균 재산 고동진 최고 신규 국회의원 평균 재산 고동진 최고 조국 이...,-0.1
5409,2

In [10]:
df_bigkinds.columns

Index(['뉴스 식별자', '일자', '언론사', '기고자', '제목', '통합 분류1', '통합 분류2', '통합 분류3',
       '사건/사고 분류1', '사건/사고 분류2', '사건/사고 분류3', '인물', '위치', '기관', '키워드',
       '특성추출(가중치순 상위 50개)', '본문', 'URL', '분석제외 여부', '제목_키워드', '제목_키워드_전처리후',
       '제목_키워드_전처리후_Okt', '제목_키워드_전처리후_Hannanum', '제목_키워드_전처리후_Kkma',
       '제목_키워드_전처리후_Komoran', '제목_키워드_전처리후_kiwi', '제목_키워드_전처리후_Komoran_명사',
       'Bert_Sentiment'],
      dtype='object')

In [11]:
df_bigkinds.to_excel('./Text_Data/bigkinds/삼성전자_정치_20170901-20240830_전처리후_Komoran_KoBERT.xlsx', index=False)

In [12]:
df_bigkinds = df_bigkinds.groupby('일자', as_index=False)['Bert_Sentiment'].mean()
# 소수점 1자리로 반올림
df_bigkinds['Bert_Sentiment'] = df_bigkinds['Bert_Sentiment'].round(1)
df_bigkinds['Bert_Sentiment'] = df_bigkinds['Bert_Sentiment'].apply(lambda x: 0 if x == -0.0 else x)
# 결과 확인
df_bigkinds

,일자,Bert_Sentiment
0,2017-09-01,-0.1
1,2017-09-02,-0.1
2,2017-09-04,0.0
3,2017-09-05,-0.1
4,2017-09-06,-0.1
...,...,...
1710,2024-08-23,-0.1
1711,2024-08-24,-0.1
1712,2024-08-26,0.1
1713,2024-08-27,0.1


In [13]:
# 감정 분석 결과 저장(일자별)
df_bigkinds.to_excel('./Text_Data/bigkinds/삼성전자_정치_20170901-20240830_전처리후_Komoran_KoBERT_daily.xlsx', index=False)

In [14]:
#KoBERT의 경우 다시 평가하는 경우 시간 소요 및 데이터가 변경되어 기존에 평가 데이터를 필터링하여 정상 데이터만 추출
import pandas as pd
# EXCEL파일 읽기
file_path = './Text_Data/bigkinds/삼성전자_정치_20170901-20240830_전처리후_Komoran_KoBERT.xlsx'
df_bigkinds = pd.read_excel(file_path, converters={'뉴스 식별자': str})  # '뉴스 식별자'는 해당 열의 실제 이름으로 대체

df_bigkinds = df_bigkinds[df_bigkinds['분석제외 여부'].isnull()]

df_bigkinds

# 그룹핑할 컬럼 리스트
group_columns = ['제목_키워드_전처리후_Komoran']

# 각 그룹에서 뉴스 식별자 컬럼의 최소값을 유지
df_bigkinds2 = df_bigkinds.groupby(group_columns, as_index=False).agg({'뉴스 식별자': 'min'})

# 필요한 컬럼만 선택하여 두 데이터프레임 병합
big_kinds_sentiment = pd.merge(
    df_bigkinds[['제목_키워드_전처리후_Komoran', '뉴스 식별자','일자','Bert_Sentiment']],
    df_bigkinds2[['제목_키워드_전처리후_Komoran', '뉴스 식별자']],
    on=['제목_키워드_전처리후_Komoran', '뉴스 식별자'],
    how='inner'
)

big_kinds_sentiment = big_kinds_sentiment[['뉴스 식별자', '일자', '제목_키워드_전처리후_Komoran', 'Bert_Sentiment']]
big_kinds_sentiment

big_kinds_sentiment.to_excel('./Text_Data/bigkinds/삼성전자_정치_20170901-20240830_전처리후_Komoran_KoBERT_filter.xlsx', index=False)

big_kinds_sentiment_avg = big_kinds_sentiment.groupby('일자', as_index=False)['Bert_Sentiment'].mean()
# 소수점 1자리로 반올림
big_kinds_sentiment_avg['Bert_Sentiment'] = big_kinds_sentiment_avg['Bert_Sentiment'].round(1)
big_kinds_sentiment_avg['Bert_Sentiment'] = big_kinds_sentiment_avg['Bert_Sentiment'].apply(lambda x: 0 if x == -0.0 else x)
# 결과 확인
big_kinds_sentiment_avg

# 감정 분석 결과 저장(일자별)
big_kinds_sentiment_avg.to_excel('./Text_Data/bigkinds/삼성전자_정치_20170901-20240830_전처리후_Komoran_KoBERT_daily_filter.xlsx', index=False)

In [15]:
big_kinds_sentiment_avg

,일자,Bert_Sentiment
0,2017-09-01,-0.1
1,2017-09-02,-0.1
2,2017-09-04,0.0
3,2017-09-05,-0.1
4,2017-09-06,-0.1
...,...,...
1648,2024-08-23,-0.1
1649,2024-08-24,-0.1
1650,2024-08-26,0.1
1651,2024-08-27,0.1


In [16]:
big_kinds_sentiment

,뉴스 식별자,일자,제목_키워드_전처리후_Komoran,Bert_Sentiment
0,20170901174354001,2017-09-01,최순 실 게이트 청문회 불 출석 11 ㄴ 중 8 명 백수 불 출석 최순 실 게이트 ...,-0.1
1,20170901141935001,2017-09-01,두문불출 10 개월 만 문고리 실세 이재만 안 봉 근 법정 나오 청문회 불 출석 인...,0.0
2,20170901153936001,2017-09-01,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,-0.2
3,20170902162415001,2017-09-02,서울 도심 곳곳 서 태극기 집회 서울 도심 태극기 집회 주말 도심 서울 도심 박근혜...,-0.1
4,20170904091746001,2017-09-04,코스 피 북 핵 리스크 2330 선 털썩 코스 피 리스크 2330 선 북한 지정학 ...,0.0
...,...,...,...,...
4945,20240829144509001,2024-08-29,재산 333 억 원 페라리 소유 국회의원 재산 신고 1 위 재산 페라리 소유 국회의...,0.0
4946,20240829000230001,2024-08-29,신규 국회의원 평균 재산 27 억 고동진 333 억 최고 신규 국회의원 평균 재산 ...,-0.1
4947,20240829070127001,2024-08-29,22 대 국회 신규 등록 최고 부자 는 333 억 고동진 조국 52 억 이준석 12...,-0.1
4948,20240829002204001,2024-08-29,333 억 고동진 22 대 초선 재산 1 위 333 억 고동진 초선 재산 국회 입성...,0.0
